# Tarea 1 - IIC2440 

### Sofía Escobedo - Matías Aedo

## Librerías

Importamos las librerías que serán utilizadas a lo largo de la tarea. Es también necesario tener instalado "wget" y "datasketch".

In [181]:
import os
import re
import csv
import pickle
import random
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output
from datasketch import MinHashLSH, MinHash

## Carga de datos

Ahora cargamos los datos; si ya se tiene el dataset no será necesario descargarlo de dropbox con wget pero se deja la opción.

In [3]:
os.listdir()

['.ipynb_checkpoints', 'T1_PDM.ipynb', 'tweets_2022_abril_junio.csv']

In [4]:
if 'tweets_2022_abril_junio.csv' in os.listdir():
    print('Dataset ya descargado.')
else:
    !wget "https://www.dropbox.com/s/j4h5v1arnemhbq3/tweets_2022_abril_junio.csv"

Dataset ya descargado.


Llevamos nuestro dataset al DataFrame con el que trabajaremos.

In [5]:
df = pd.read_csv('tweets_2022_abril_junio.csv')

## Visualización inicial

Realizamos una visualización inicial del dataframe para ver con qué estamos trabajando.

In [6]:
df.head()

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0


También queremos saber con cuántas filas estamos trabajando en un principio.

In [7]:
shape_0 = df.shape
print(f"El dataframe tiene {shape_0[0]} filas.")

El dataframe tiene 4594980 filas.


## Preprocesamiento y limpieza de datos

Primero buscaremos eliminar filas duplicadas y valores nulos.

In [8]:
df = df.drop_duplicates(subset=['id', 'text'])
df = df.dropna(subset=['id', 'text', 'screen_name'])
shape_1 = df.shape
print(f"Esta acción se deshizo de {shape_0[0] - shape_1[0]} filas. \nEl dataframe tiene {shape_1[0]} filas.")

Esta acción se deshizo de 2174 filas. 
El dataframe tiene 4592806 filas.


Tampoco nos interesa mantener aquellas filas cuyo "screen_name" y "text" es el mismo, es decir, un usuario que twitteó dos veces lo mismo.

In [9]:
df = df.drop_duplicates(subset=['screen_name', 'text'])
shape_2 = df.shape
print(f"Esta acción se deshizo de {shape_1[0] - shape_2[0]} filas. \nEl dataframe tiene {shape_2[0]} filas.")

Esta acción se deshizo de 42834 filas. 
El dataframe tiene 4549972 filas.


Entre los tweets que vimos en la visualización inicial pudimos notar que son todos retweets. Ya que los retweets de una persona no nos dicen nada sobre su propio estilo de escritura, hemos decidido eliminarlos.

In [10]:
df = df[~df['text'].str.startswith('RT')]
shape_3 = df.shape
print(f"Esta acción se deshizo de {shape_2[0] - shape_3[0]} filas. \nEl dataframe tiene {shape_3[0]} filas.")

Esta acción se deshizo de 3299423 filas. 
El dataframe tiene 1250549 filas.


Ahora que terminamos el preprocesamiento de los datos, pasaremos a la limpieza del "texto" de los tweets. Usaremos una copia del dataframe para mostrar ciertos ejemplos de la limpieza que se hará luego.

In [11]:
df_limpieza = df.copy()

Pudimos notar que en lugar de "&" se encontraba la presencia de "&amp" en los tweets. 

A continuación, mostramos un ejemplo de un tweet en el que esto sucede y cómo quedará después de limpiarlo.

In [12]:
ejemplo_1 = df_limpieza.text[4549723]
print("Tweet previo a limpieza:\n", ejemplo_1)
ejemplo_1 = BeautifulSoup(ejemplo_1, 'lxml')
ejemplo_1 = ejemplo_1.get_text()
df_limpieza.at[4549723, 'text'] = ejemplo_1
print("Tweet posterior a limpieza:\n", df_limpieza.text[4549723])

Tweet previo a limpieza:
 @mcubillossigall &amp; fuiste ministra de educación JAJASJAJ 🤡🤡🤡🤡🤡
Tweet posterior a limpieza:
 @mcubillossigall & fuiste ministra de educación JAJASJAJ 🤡🤡🤡🤡🤡


Muchos de los tweets tienen imágenes o gifs, la mayoría utilizadas como "reacciones" o "memes". El mantener esto en el "texto" del tweet podría causar problemas a la hora de encontrar personas similares, ya que no tiene que ver con su estilo de escritura y podría sezgar los resultados a simplemente personas que utilizaron la misma imagen de reacción. 

Por esto, serán eliminados los "links" que denotan imágenes o gifs. A continuación, mostramos un ejemplo de un tweet en el que esto sucede y cómo quedará después de limpiarlo.

In [13]:
ejemplo_2 = df_limpieza.text[4568847]
print("Tweet previo a limpieza:\n", ejemplo_2)
ejemplo_2 = re.sub('https?://[A-Za-z0-9./]+','',ejemplo_2)
df_limpieza.at[4568847, 'text'] = ejemplo_2
print("Tweet posterior a limpieza:\n", df_limpieza.text[4568847])

Tweet previo a limpieza:
 @christianpviera Taba juerte parece. https://t.co/QvdsK7OeGm
Tweet posterior a limpieza:
 @christianpviera Taba juerte parece. 


Al realizar el resto del proceso con la limpieza tal cual está hasta el momento, notamos que comenzaba a agrupar los tweets similares según las menciones realizadas en un tweet, por lo que cualquier respuesta o mención a otra persona contaba como una similaridad. Esto sezga los resultados, ya que no encuentra similitudes según el estilo de escritura sino según a qué persona se menciona.

Por esto, decidimos eliminar las menciones. A continuación, mostramos un ejemplo de un tweet en el que esto sucede y cómo quedará después de limpiarlo.

In [14]:
ejemplo_3 = df_limpieza.text[17]
print("Tweet previo a limpieza:\n", ejemplo_3)
ejemplo_3 = ejemplo_3.replace("_", "")
ejemplo_3 = re.sub(r'@\w+\b','', ejemplo_3)
ejemplo_3 = ejemplo_3.lstrip()
df_limpieza.at[17, 'text'] = ejemplo_3
print("Tweet posterior a limpieza:\n", df_limpieza.text[17])

Tweet previo a limpieza:
 @HernanLarrain @gabrielboric Que va a hacer nada!! Solo prestarle ropa para que se siga equivocando
Tweet posterior a limpieza:
 Que va a hacer nada!! Solo prestarle ropa para que se siga equivocando


Ahora que ya se han mostrado ejemplos de la limpieza que será realizada, es momento de realizarla sobre todas las filas del dataframe preprocesado. Para distintas pruebas que se quieren realizar, los tweets serán divididos según su largo con los grupos siendo tweets de entre 0 a 70 caracteres, 71 a 120 caracteres y de más de 120 caracteres.

In [18]:
%%time
tweets_0_70 = {}
tweets_71_120 = {}
tweets_121_x = {}

if 'tweets_0_70_noRT.pkl' in os.listdir():
    file = open('tweets_0_70_noRT.pkl', 'rb')
    tweets_0_70 = pickle.load(file)
    file.close()

    file = open('tweets_71_120_noRT.pkl', 'rb')
    tweets_71_120 = pickle.load(file)
    file.close()

    file = open('tweets_121_x_noRT.pkl', 'rb')
    tweets_121_x = pickle.load(file)
    file.close()
else:
    for index, row in df.iterrows():
        texto = row['text']
      
        if '&amp' in texto:
            texto = BeautifulSoup(texto, 'lxml')
            texto = texto.get_text()
            df.at[index, 'text'] = texto

        if 't.co' in texto:
            texto = re.sub('https?://[A-Za-z0-9./]+', '', texto)
            df.at[index, 'text'] = texto
      
        if '@' in texto:
            texto = texto.replace("_", "")
            texto = re.sub(r'@\w+\b','', texto)
            texto = texto.lstrip()
            df.at[index, 'text'] = texto
        
        if len(texto) > 2:
            if len(texto) < 71:
                tweets_0_70[str(row['id'])] = texto
            elif len(texto) < 121:
                tweets_71_120[str(row['id'])] = texto
            else:
                tweets_121_x[str(row['id'])] = texto
                
    df = df[df['text'].str.len()>2]
        
    file = open('tweets_0_70_noRT.pkl', 'wb')
    pickle.dump(tweets_0_70, file)
    file.close()

    file = open('tweets_71_120_noRT.pkl', 'wb')
    pickle.dump(tweets_71_120, file)
    file.close()

    file = open('tweets_121_x_noRT.pkl', 'wb')
    pickle.dump(tweets_121_x, file)
    file.close()
            

Wall time: 1min 12s


In [20]:
print(f"Cantidad de tweets entre 0 y 70 caracteres: {len(tweets_0_70)}")
print(f"Cantidad de tweets entre 71 y 120 caracteres: {len(tweets_71_120)}")
print(f"Cantidad de tweets con más de 120 caracteres: {len(tweets_121_x)}")

Cantidad de tweets entre 0 y 70 caracteres: 834150
Cantidad de tweets entre 71 y 120 caracteres: 276811
Cantidad de tweets con más de 120 caracteres: 10684


Se creará "big_tweets" como unión de los tres grupos anteriores para el procesamiento de datos con todas las filas juntas.

In [21]:
big_tweets = tweets_0_70 | tweets_71_120 | tweets_121_x

print(f"Cantidad de tweets total: {len(big_tweets)}")

Cantidad de tweets total: 1121645


## Funciones de comparación, hashing y Jaccard

Función para generar k-shingles a partir de un texto dado.

In [24]:
def generate_k_shingles(text, k):
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i+k]
        shingles.add(shingle)
    return shingles

Función para crear un índice LSH a partir de un diccionario de textos, utilizando el algoritmo de firma MinHash y el esquema LSH  a las firmas.

In [25]:
def create_hash_lsh(texts_dict, threshold, k):
    lsh = MinHashLSH(threshold=threshold)
    count = 0
    for text_id, text in texts_dict.items():
        shingles = generate_k_shingles(text, k)
        minhash = MinHash()

        for shingle in shingles:
            minhash.update(shingle.encode('utf8'))
        
        lsh.insert(text_id, minhash)
        count += 1
        
        # descomentar si se quieren "updates" 
        #if count % 5000 == 0:
        #    print("Se han hasheado ", count, " tweets")

    return lsh

Función para buscar en un índice LSH los elementos que coinciden con un tweet dado.

In [26]:
def obtain_bucket(lsh, tweet, k):
    shingles = generate_k_shingles(tweet, k)
    minhash_actual = MinHash()
    for shingle in shingles:
        minhash_actual.update(shingle.encode('utf8'))

    result = lsh.query(minhash_actual)
    return result

Función para calcular el coeficiente de Jaccard entre dos tweets dados.

In [27]:
def jaccard(tweet1, tweet2, k):
    shingles1 = generate_k_shingles(tweet1, k)
    shingles2 = generate_k_shingles(tweet2, k)

    result = len(shingles1.intersection(shingles2)) / len(shingles1.union(shingles2))

    return result

Función para buscar tweets similares a uno de referencia utilizando un índice LSH y el coeficiente de Jaccard. Si se ingresa un "reference_id" se buscan para el tweet pedido, y si no se busca un tweet al azar dentro del diccionario text_dict.

In [150]:
def get_similar_tweets(text_dict, reference_id = False, print_it = False):
    if not reference_id:
        reference_id, reference_text = random.choice(list(text_dict.items()))
    else:
        reference_text = text_dict[reference_id]
    
    # obtener el bucket del tweet con sus tweets similares según lsh
    bucket = obtain_bucket(lsh, reference_text, k)
    
    # evaluar su similaridad según el coeficiente de Jaccard
    tweets_similares = {}
    for id in bucket:
        similaridad = jaccard(reference_text, text_dict[id], k)
        if similaridad > s and reference_id!=id:
            tweets_similares[id] = [text_dict[id], similaridad]
    
    # imprimir los tweets si es requerido
    if print_it:
        print(f'Buscando tweets similares a "{reference_text}"')
        i = 1
        for key, value in tweets_similares.items():
            print(f'{i}. {value[0]}')
            i += 1
            
    return [reference_id, tweets_similares]

Función para guardar los tweets similares en un csv.

In [176]:
def save_dict_to_csv(dictionary, filename):
    with open(filename, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'Tweet'])  # Escribir el encabezado

        for key, value in dictionary.items():
            writer.writerow([key, value[0]]) # Escribir cada fila

Función para ajustar el diccionario "usuarios" según tweets similares a un tweet específico.

In [65]:
def tweet_similar_users(df, tweets_similares, usuarios):
    for key, value in tweets_similares.items():
        user = df.loc[df['id'] == int(key), 'screen_name'].values[0]
        if user in usuarios.keys():
            cantidad = usuarios[user][1]
            new_value = ((usuarios[user][0] * cantidad) + value[1])/(cantidad+1)
            usuarios[user] = [new_value, cantidad+1]
        else: 
            usuarios[user] = [value[1], 1]
    return usuarios

Función para truncar la lista de usuarios similares, quedandonos solo con aquellos que tengan más de "t" tweets similares y por sobre un "s" de similaridad en general.

In [147]:
def most_similar_users(usuarios, s, t):
    u_copy = usuarios.copy()
    for u in u_copy:
        if usuarios[u][0] < s:
            del usuarios[u]
        elif usuarios[u][1] < t:
            del usuarios[u]
            
    usuarios = {k: v for k, v in sorted(usuarios.items(), key=lambda x: x[1][0], reverse=True)}
    return usuarios

Función para encontrar usuarios similares a uno en específico; para hacer esto se buscan todos los tweets de ese usuario y para cada uno sus tweets similares. Con esto se va creando el diccionario "usuarios" cuya llave es el nombre del otro usuario y sus valores son la similaridad promedio y la cantidad de tweets similares encontrados. 

Para finalizar, se truncan los resultados para obtener aquellos usuarios que tengan más de 10 tweets similares (ya que consideramos que menos de eso es inconclusivo) y más de un 0.5 de similaridad (ya que consideramos que menos de eso no es realmente similar). 

Si no se entrega un usuario, la función buscará uno al azar.

In [165]:
def get_similar_users(df, tweets_similares, text_dict, user=False):
    if not user:
        user = df.sample().values[0][2]
    
    print(f'Buscando usuarios similares a {user}...')
    
    user_tweets = df[df['screen_name'] == user]
    
    i = 1
    usuarios = {}
    for tweet in user_tweets.values:
        print(f'Revisando {i}/{len(user_tweets)} tweets...')
        reference_id, tweets_similares = get_similar_tweets(big_tweets, str(tweet[0]))
        usuarios = tweet_similar_users(df, tweets_similares, usuarios)
        i += 1
    
    usuarios = most_similar_users(usuarios, 0.5, 10)
    
    if len(usuarios) == 0:
        print('No se lograron encontrar usuarios similares.')
    return [user, usuarios]

Función para imprimir los usuarios similares a un usuario.

In [124]:
def print_similar_users(usuario, usuarios):
    print(f'Los usuarios más similares a {usuario} son')
    i = 1
    for u in usuarios:
        print(f'{i}. {u} con una similaridad de {usuarios[u][0]} para {usuarios[u][1]} tweets.')
        i += 1 

Función para imprimir los perfiles de dos usuarios similares encontrados.

In [145]:
def show_similar_pair(user1, user2):
    user1_tweets = df[df['screen_name'] == user1]
    user2_tweets = df[df['screen_name'] == user2]
    
    print(f'Perfil del Usuario {user1}')
    for tweet in user1_tweets.values:
        print(tweet[3])
    
    print('- '*30)
    
    print(f'Perfil del Usuario {user2}')
    for tweet in user2_tweets.values:
        print(tweet[3])

## Procesamiento de datos

Creamos el LSH. Se puede probar con distintos k, s y threshold. Esta celda toma entre 20 y 40 minutos.

In [33]:
k = 3
s = 0.2
threshold = 0.2

# Se crea el LSH
lsh = create_hash_lsh(big_tweets, threshold, k)

## Obtener tweets similares

A continuación, entregamos dos ejemplos obteniendo tweets similares a uno dado. Primero será con uno que hayamos elegido, y el otro será al azar.

In [179]:
similar_tweets = get_similar_tweets(big_tweets, '1525861915976556550', True)

Buscando tweets similares a "Metete tus normas donde ud sabe"
1. Metete tu solidaridad por donde te quepa
2. Ya sabes donde meterte tus "buenas noticias" para ti, con letra chica para el resto.
3. Metete tus normas por el orto pajaron qliao
4. Ni si quiera  se sabe las normas.
5. Este no sabe donde tiene la #raja !!!
6. Metete tu borrador por donde mas te guste💩💩💩🤬🤬🤬🤬🤬
7. Una Mier… tus normas!!! 
8. Evidentemente no sabes donde estás parada
9. Ninguno, ud no sabe entender las normas
10. Puro pico tus normas saco wueas
11. Métete tu borrador por donde tu sabes.
12. Metete tu porquería por donde te quepa
13. Metete tu CC por donde te quepa!!!!
14. Metete tus palabras por donde quieras.
15. De donde sacó ud eso?
16. Metete tu pluri por donde mas te quepa vieja chanta
17. Metela ya sabes por donde
18. Metas las normas donde le quepan...
19. 🤣🤣🤣🤣🤣 ya sabes donde meterte la carta 
20. Metetela donde te caiga😏
21. Metetela x donde te quepa ctm
22. Sabes donde metertetela 
23. Metete las Cápsu

Ahora lo guardamos en csv.

In [182]:
save_dict_to_csv(similar_tweets[1], 'tweets_similares_1.csv')

Probamos con un tweet generado al azar.

In [183]:
similar_tweets = get_similar_tweets(big_tweets, print_it=True)

Buscando tweets similares a "Hey 👋 ¡Ya es hora de que te recuerde lo que me has pedido!
"
1. Un recuerdo de lo que han hecho los angelitos 
2. Y que es lo que te decepciona de la CC??
3. Le recuerdo que la pareja de Medel es Española.
4. Es decir, no lo harán, te recuerdo que en campaña era una de las prioridades de Boric 
5. , para recuerdes parece que la memoria te está fallando!.
6. A mi no me da lo mismos...y te recuerdo que voto.
7. No tienes idea de lo que hablas pelmazo ql
8. Se supone que es parte de lo que tienen que hacer?? O me equivoco??
9. No es peor de lo que hacen los zurdos.
10. ¡Hey! ¡Te recuerdo lo que me solicitaste!

11. Es hora de que dejes de twittear y hagas la pega
12. La esperanza de rechazar es lo último que perderemos.
13. Tu lo que menos tienes es conciencia de lo que hablas
14. Es hora de que les reviente a todos ustedes progres de mierda
15. Y para que te acuerdas de mierdas 💩
16. di la verdad lo que te molesta es los derechos de agua
17. Señora deje de te

Y lo guardamos en csv.

In [184]:
save_dict_to_csv(similar_tweets[1], 'tweets_similares_2')

## Obtener personas similares

A continuación, entregamos tres ejemplos obteniendo personas similares a un usuario dado. Primero será con uno que hayamos elegido porque tienen estilos de escritura similar, el segundo será uno que elegimos por su contenido similar, y el otro será al azar.

Buscamos usuarios similares a "jnfernandez" utilizando la función ya discutida "get_similar_users".

In [170]:
user, users = get_similar_users(df, tweets_similares, big_tweets, 'jnfernandez')
clear_output()

Imprimimos los usuarios más similares a "jnfernandez" junto a su similaridad promedio y la cantidad de tweets similares en total.

In [125]:
print_similar_users(user, users)

Los usuarios más similares a jnfernandez son
1. pennaleon con una similaridad de 0.7000000000000001 para 10 tweets.
2. Ninoskadamianov con una similaridad de 0.609831029185868 para 16 tweets.
3. ChileDemocrata con una similaridad de 0.5951284861770796 para 10 tweets.
4. SoyLaIngo con una similaridad de 0.5932539682539683 para 14 tweets.
5. 711Diaz con una similaridad de 0.5563687724704675 para 11 tweets.
6. santosfreaks con una similaridad de 0.5347939885173929 para 12 tweets.
7. Nathalie_lav con una similaridad de 0.5330184435447592 para 10 tweets.
8. PatricioOgalde5 con una similaridad de 0.5125 para 12 tweets.


Mostramos el perfil del usuario con el que partimos y su usuario más "similar". Se puede ver que a pesar de que uno es claramente por el rechazo y el otro es del apruebo, los dos son muy agresivos en su estilo de escribir.

In [146]:
show_similar_pair(user, list(users.keys())[0])

Perfil del Usuario jnfernandez
Poniendo una chaucha (10%) y te entragan una tasa de reemplazo del 30%. Q más quieres. Pura ignorancia estos.
Uno puede seguir el juego, pero nadie espera q hagan una piñata con tu cara para hacerla tira. No sea weon!
Y dices: Deja la pasta.  Es no cachar nada.
Se entiende lo picante q es el mamarracho.
Pídele a tu presidente q apoye el 100%  o  a los pasteles de la CC. Veamos si somos o no dueños... IGNORANTE!
Esa es la pelea, lo saben y es tramposo. Ya lo hicieron para la cc.
Ya se puso grosera y no entiendes el punto. Dije: gente fresca.
Para q dejarlo en manos de políticos pencas??? Entiende el punto.
Q lamentable. Algo oscuros los atrias. Arica sabe bien de 
Jajaja le acomodas el pillama? Da lo mismo.
Te salió el cuco.
jajajaja date cuenta.
Debe ser los tipos más lucidos en esa wea.
Copy paste de la venezolana.  Te faltó: en justicia.
Q lea algo d lo q pasa en el sur y en el norte.
Aahh pero para lo que usan piñata con la cara de otros... defiendes. 

Buscamos usuarios similares a "CarlosEsDelAlbo" utilizando la función ya discutida "get_similar_users".

In [171]:
user, users = get_similar_users(df, tweets_similares, big_tweets, 'CarlosEsDelAlbo')
clear_output()

Imprimimos los usuarios más similares a "jnfernandez" junto a su similaridad promedio y la cantidad de tweets similares en total.

In [173]:
print_similar_users(user, users)

Los usuarios más similares a CarlosEsDelAlbo son
1. EsVictorHugo con una similaridad de 0.887719298245614 para 14 tweets.
2. LorenaTzoc con una similaridad de 0.845022624434389 para 26 tweets.
3. vrojas007 con una similaridad de 0.80344409027979 para 30 tweets.
4. Paomotion con una similaridad de 0.7493578723834563 para 10 tweets.
5. AlvaroBots_ con una similaridad de 0.7455341650830373 para 14 tweets.
6. soniadederecha con una similaridad de 0.7373641278303645 para 58 tweets.
7. LourdesGavar con una similaridad de 0.7333333333333334 para 10 tweets.
8. bybleanstartup con una similaridad de 0.73306887770365 para 41 tweets.
9. fgb_sw con una similaridad de 0.730786886807141 para 14 tweets.
10. rodvalpablo con una similaridad de 0.7267904509283819 para 12 tweets.
11. Jo5836613057 con una similaridad de 0.7059702554614673 para 12 tweets.
12. AngelDLaCruz_ con una similaridad de 0.6953763671717832 para 18 tweets.
13. SuknoR con una similaridad de 0.6931773879142301 para 10 tweets.
14. PATEF

Mostramos el perfil del usuario con el que partimos y su usuario más "similar". Se puede ver que ambos tienen contenido similar del Rechazo.

In [174]:
show_similar_pair(user, list(users.keys())[0])

Perfil del Usuario CarlosEsDelAlbo
El vuelo nunca fuea brea culiao
Lo de la libertad pasó piola, se mandó dos cagazos en una frase de 1 minuto
Pero si Lagos ya lo dijo, si gana rechazo sigue la constitución que hizo él
Puras felicitaciones, como decía Mañalich hajajaj
Tenía que salir una defensora de los narcos del sur
Y lo malo es que no le pasará nada, Adrian tiene mucha influencia desde el gobierno de Bachelet
Se debe ir a la chucha, mínimo, pero primero acusación constitucional
Se quedó callado el zurdo
El que lo necesita es porque no se sacó debidamente la chucha trabajando
Con los impuestos se financia a cada vago del congreso y la moneda, tampoco deberíamos pagarlos
Aunque te duela el hoyo vieja culia 
Tiene razón y eso que voy por el Rechazo, si se aprueba el Mamarracho deben haber elecciones nuevas
Si como se regula hasta ahora por el norte
Dile a los borrachos pasteros de la convención que dejen de hacerse autogoles, #ElRechazoCrece
Es el pelao culiao de las series
Le dan has

Buscamos usuarios similares a un usuario generado al azar utilizando la función ya discutida "get_similar_users".

In [185]:
user, users = get_similar_users(df, tweets_similares, big_tweets)
clear_output()

Imprimimos los usuarios más similares a este usuario al azar junto a su similaridad promedio y la cantidad de tweets similares en total.

In [186]:
print_similar_users(user, users)

Los usuarios más similares a lorenzojavier7 son
1. ONJ999 con una similaridad de 1.0 para 14 tweets.
2. GabrielRuizT con una similaridad de 1.0 para 11 tweets.
3. CoachRVargasV con una similaridad de 1.0 para 10 tweets.
4. Alejand81158101 con una similaridad de 1.0 para 10 tweets.
5. Alvar38065484 con una similaridad de 0.9666666666666667 para 15 tweets.
6. manuelsoteldo4 con una similaridad de 0.9013238916256158 para 16 tweets.
7. AprendizCrypto con una similaridad de 0.8547619047619047 para 10 tweets.
8. VeneciaSU con una similaridad de 0.8496376811594203 para 12 tweets.
9. albert_1917 con una similaridad de 0.8263888888888888 para 12 tweets.
10. Mister_Wolf_0 con una similaridad de 0.75 para 11 tweets.
11. JorgeMi62044105 con una similaridad de 0.7015873015873015 para 36 tweets.
12. volffit con una similaridad de 0.6862369337979095 para 10 tweets.
13. MarcelaBH201 con una similaridad de 0.5339602925809822 para 11 tweets.


Mostramos el perfil del usuario con el que partimos y su usuario más "similar".

In [187]:
show_similar_pair(user, list(users.keys())[0])

Perfil del Usuario lorenzojavier7
Maravillosa respuesta
En Chile hay gente armada que asesina Chilenos de ascendencia mapuche impunemente. 
Y a los asesores sin límite de remuneración
Que derecho tiene para ley de amarre. NINGUNO
Dictadura encubierta
Que los juzguen
Y someter por la sed a cualquier usuario
La maquina ideológica totalitaria es  previa y la amenaza de inestabilidad si gana rechazo que es? 🤔
Pero si el Estado no es un producto Machista HeteroPatriarcal. ¿ Y quiere más Estado? Algo no cuadra. 
Cuanta soberbia y fakenews aquí la avidez por los ahorros de los trabajadores 
Eso no es educación es adoctrinamiento o como se decía en regímenes totalitarios reeducacion.
Siempre el empate, se supone que ellos iban a renovar la política...
Yaaa lo que el mundo quiere abandonar es rl esquema ponzi del reparto
No es más que obsesión por la verdad y no por eufemismos y medias mentiras
Y los desorbitados gastos de Punlicidad $1.300 millones y para IFE no hay 
Se omiten tantas cosas y l